In [1]:
## mounting the google drive in order to access the .json file for dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Installing the neccessary libraries
!pip install transformers
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 855.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.0 MB/s eta 0:00:

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name) ## Loading the pre-trained GPT-2 language model
tokenizer = GPT2Tokenizer.from_pretrained(model_name) ## Loading the pre-trained GPT-2 language model

In [4]:
## Importing all the necessary libraries
import json
import torch
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm

In [5]:
path = "/content/drive/MyDrive/LS_final_project/chat_data.json"
a = json.load(open(path, "r")) ## Loading the dataset which is a .json file
a[0]

{'dialog_id': '0xab38710',
 'dialog': [{'id': 0,
   'sender': 'participant2',
   'text': 'I love iphone! i just bought new iphone!'},
  {'id': 1,
   'sender': 'participant1',
   'text': "Thats good for you, i'm not very into new tech"},
  {'id': 2,
   'sender': 'participant2',
   'text': 'I am a college student and i am a college student'},
  {'id': 3,
   'sender': 'participant1',
   'text': 'I am go to gym and live on donations'},
  {'id': 4,
   'sender': 'participant2',
   'text': 'I am a vegan and i am in the midwest'},
  {'id': 5,
   'sender': 'participant1',
   'text': 'So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat'},
  {'id': 6,
   'sender': 'participant2',
   'text': 'I would not mind having them in the office that would be hard for me'},
  {'id': 7, 'sender': 'participant1', 'text': 'Dogs or vegan in office?'},
  {'id': 8,
   'sender': 'participant2',
   'text': 'I am a vegetarian so i am vegan'},
  {'id': 9, 'sender': 'participa

In [6]:
len(a) ## each a[i] has a dialog in which there is a text we want that text only

427

In [11]:
class chatbot(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = json.load(open(path, "r")) ## Load the data

        self.X = [] ## Initialize a list to store the dialog texts
        for i in self.data: ## Iterate over each item in the loaded data
            for j in i['dialog']:
                self.X.append(j['text']) ## Append the text from the dialog to the X list
        for idx, i in enumerate(self.X):
            try:
                ## Modify the text to include context and bot responses
                self.X[idx] = "<startofstring> "+i+" <bot>: "+self.X[idx+1]+" <endofstring>"
            except:
                break
        self.X = self.X[:10000]
        print(self.X[:5])

        ## Tokenizing the modified X list using the provided tokenizer
        self.X_encoded = tokenizer(self.X, max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        ## Taking the max_length=60 will truncate all X to length 60 by either adding special tokens or by cutting it to length 60
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X) ## Return the number of elements in the dataset

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx]) ## Return a tuple containing the input IDs and attention mask for a given index

In [12]:
## added special tokens to the tokenizer's vocabulary that we used in the dataset
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

0

In [13]:
## making the dataset for model training
chatbot = chatbot(path, tokenizer)
chatbot = DataLoader(chatbot, batch_size=64)

["<startofstring> I love iphone! i just bought new iphone! <bot>: Thats good for you, i'm not very into new tech <endofstring>", "<startofstring> Thats good for you, i'm not very into new tech <bot>: I am a college student and i am a college student <endofstring>", '<startofstring> I am a college student and i am a college student <bot>: I am go to gym and live on donations <endofstring>', '<startofstring> I am go to gym and live on donations <bot>: I am a vegan and i am in the midwest <endofstring>', '<startofstring> I am a vegan and i am in the midwest <bot>: So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat <endofstring>']


In [ ]:
chatbot.size

In [10]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

In [20]:
## Function to generate responses from the chatbot model given an input

def infer(inp):
    ## Preprocessing text
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)

    ## generating the model output and decoding it
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])

    return output

In [17]:
## function to train the model
def train(chatbot, model, optim):

    epochs = 12

    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatbot:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad() ## Clearing the gradients from the previous iteration
            loss = model(X, attention_mask=a, labels=X).loss  ## Forward pass
            loss.backward() ## Backpropagate the gradients
            optim.step() ## updating parameters
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("hello how are you"))

In [18]:
model.resize_token_embeddings(len(tokenizer)) ## resizing the token embeddings of the model to match the vocabulary size of the tokenizer
model = model.to(device) ## As we added special tokens we have to resize it
print(model.train())
optimizer = Adam(model.parameters(), lr=1e-3) ## Initializing the optimizer

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50261, bias=False)
)


In [19]:
train(chatbot, model, optimizer) ## Training our model on the chatbot dataset

  0%|          | 0/12 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  8%|▊         | 1/12 [00:44<08:12, 44.74s/it]

<startofstring> hello how are you <bot>: <bot>: <bot>: <bot>: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 17%|█▋        | 2/12 [01:24<06:57, 41.78s/it]

<startofstring> hello how are you <bot>: <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 25%|██▌       | 3/12 [02:04<06:06, 40.76s/it]

...

<startofstring> hello how are you <bot>: i am a huge gamer <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 92%|█████████▏| 11/12 [07:26<00:40, 40.48s/it]

<startofstring> hello how are you <bot>: Hello <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 12/12 [08:05<00:00, 40.47s/it]

<startofstring> hello how are you <bot>: Hi, how are doing? <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [22]:
torch.save(model.state_dict(), "chatbot_model.pth") ## saving the model

In [24]:
import shutil
shutil.copy("chatbot_model.pth", "/content/drive/MyDrive/LS_FINAL_SUBMISSION")

'/content/drive/MyDrive/LS_FINAL_SUBMISSION/chatbot_model.pth'

In [27]:
infer("Who are you ?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'<startofstring> Who are you? <bot>: I am a huge gamer <endofstring> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>'